In [1]:
from line_solver import *

In [2]:
model = Network('model')

P1 = Place(model, 'P1')
P2 = Place(model, 'P2')
P3 = Place(model, 'P3')

T1 = Transition(model, 'T1')
T2 = Transition(model, 'T2')
T3 = Transition(model, 'T3')
T4 = Transition(model, 'T4')

jobclass = ClosedClass(model, 'Class1', 8, P1, 0)

In [3]:
# T1 - Competes for tokens: requires 2 tokens from P1, produces 2 to P2
mode1 = T1.add_mode('Mode1')
T1.set_distribution(mode1, Exp(2))
T1.set_enabling_conditions(mode1, jobclass, P1, 2)
T1.set_firing_outcome(mode1, jobclass, P2, 2)

# T2 - Competes for tokens: requires 3 tokens from P1, produces 3 to P3
mode2 = T2.add_mode('Mode2')
T2.set_distribution(mode2, Exp(1))
T2.set_enabling_conditions(mode2, jobclass, P1, 3)
T2.set_firing_outcome(mode2, jobclass, P3, 3)

# T3 - Returns tokens: takes 1 from P2, returns 1 to P1
mode3 = T3.add_mode('Mode3')
T3.set_distribution(mode3, Exp(4))
T3.set_enabling_conditions(mode3, jobclass, P2, 1)
T3.set_firing_outcome(mode3, jobclass, P1, 1)

# T4 - Returns tokens: takes 2 from P3, returns 2 to P1
mode4 = T4.add_mode('Mode4')
T4.set_distribution(mode4, Exp(2))
T4.set_enabling_conditions(mode4, jobclass, P3, 2)
T4.set_firing_outcome(mode4, jobclass, P1, 2)

In [4]:
# Set up routing matrix
routingMatrix = model.init_routing_matrix()

# Place to transition routing (competing transitions from P1)
routingMatrix.set(jobclass, jobclass, P1, T1, 1.0)  # P1 -> T1
routingMatrix.set(jobclass, jobclass, P1, T2, 1.0)  # P1 -> T2 (competing)
routingMatrix.set(jobclass, jobclass, P2, T3, 1.0)  # P2 -> T3
routingMatrix.set(jobclass, jobclass, P3, T4, 1.0)  # P3 -> T4

# Transition to place routing
routingMatrix.set(jobclass, jobclass, T1, P2, 1.0)  # T1 -> P2
routingMatrix.set(jobclass, jobclass, T2, P3, 1.0)  # T2 -> P3
routingMatrix.set(jobclass, jobclass, T3, P1, 1.0)  # T3 -> P1
routingMatrix.set(jobclass, jobclass, T4, P1, 1.0)  # T4 -> P1

model.link(routingMatrix)

In [5]:
# Set initial state - all 8 tokens start in P1
P1.set_state([jobclass.get_population()])
P2.set_state([0])
P3.set_state([0])

In [6]:
# Solve the model
solver = JMT(model, seed=23000)
avgTable = solver.avg_table()

Station JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
     P1   Class1   0.0   0.0    0.0     0.0   0.0   0.0
     P2   Class1   0.0   0.0    0.0     0.0   0.0   0.0
     P3   Class1   0.0   0.0    0.0     0.0   0.0   0.0
